In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test

In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
from math import log
from CIoTS import *
import json
from time import time
import random

In [15]:
runs = 5
p_range = [5, 10, 15, 20] 
dimensions = 3
data_length = 10000
alpha = 0.05
in_edges_range = [2, 4]
auto_corr = 0.75
patience = 2*max(p_range)
p_max = 2*max(p_range)

In [16]:
results = []
for incoming_edges in in_edges_range:
    for p in p_range:
        setting = {'dim': dimensions, 'p': p, 'in_edges': incoming_edges, 'auto_corr': auto_corr,
                   'patience': patience, 'p_max': p_max, 'alpha': alpha, 'experiments': []}
        for run in range(runs):
            generator = CausalTSGenerator(dimensions=dimensions, max_p=p, data_length=data_length,
                                          incoming_edges=incoming_edges, autocorrelation=auto_corr)
            ts = generator.generate()
            
            rand_suffix = random.randint(0, 2e16-1)
            csv_name = f'data/{p}_{dimensions}_{incoming_edges}_{run}_{rand_suffix}.csv'
            experiment = {'true_graph': nx.to_dict_of_lists(generator.graph), 'data': csv_name, 'iterations': []}
            
            _, graphs, _, bics = pc_incremental(partial_corr_test, ts, alpha, p_max, 
                                                verbose=True, patiency=patience)
            for p_iter, graph in graphs.items():
                experiment['iterations'].append({'p_iter': p_iter, 'bic': bics[p_iter],
                                                 'graph': nx.to_dict_of_lists(graph)})
            
            ts.to_csv('results/'+csv_name, index=False)
            setting['experiments'].append(experiment)
            
        print(f'{incoming_edges} {p} done')
        
        results.append(setting)
        with open('results/edge_deletion.json', 'w') as f:
            json.dump(results, f)
        

2 5 0
2 5 1
2 5 2
2 5 3
2 5 4
2 10 0
2 10 1
2 10 2
2 10 3
2 10 4
2 15 0
2 15 1
2 15 2
2 15 3
2 15 4
2 20 0
2 20 1
2 20 2
2 20 3
2 20 4
4 5 0
4 5 1
4 5 2
4 5 3
4 5 4
4 10 0
4 10 1
4 10 2
4 10 3
4 10 4
4 15 0
4 15 1
4 15 2
4 15 3
4 15 4
4 20 0
4 20 1
4 20 2
4 20 3
4 20 4


In [20]:
with open('results/edge_deletion.json', 'r') as f:
    results = json.load(f)

for setting in results:
    for experiment in setting['experiments']:
        experiment['iterations'] = []
        ts = pd.read_csv('results/' + experiment['data'])

        _, graphs, _, bics = pc_incremental_extensive(partial_corr_test, ts, setting['alpha'], setting['p_max'], 
                                                      verbose=True, patiency=setting['patience'])
        for p_iter, graph in graphs.items():
            experiment['iterations'].append({'p_iter': p_iter, 'bic': bics[p_iter],
                                             'graph': nx.to_dict_of_lists(graph)})
    incoming_edges = setting['in_edges']
    p = setting['p']
    print(f'{incoming_edges} {p} done')

with open('results/edge_deletion_extensive.json', 'w') as f:
    json.dump(results, f)

4 5 done
4 10 done
4 15 done
4 20 done
4 5 done
4 10 done
4 15 done
4 20 done


In [ ]:
with open('results/edge_deletion.json', 'r') as f:
    results = json.load(f)

for setting in results:
    setting['max_cond'] = 5
    for experiment in setting['experiments']:
        experiment['iterations'] = []
        ts = pd.read_csv('results/'+experiment['data'])

        _, graphs, times, bics, sepsets, cond_sizes = pc_incremental_pc1(
            partial_corr_test, ts, setting['alpha'], setting['p_max'], 
            verbose=True, patiency=2, max_cond=5)
        experiment['sepsets'] = [{'from': k[0], 'to': k[1], 'sepset': v} for k,v in sepsets.items()]
        experiment['cond_sizes'] = cond_sizes
        for p_iter, graph in graphs.items():
            experiment['iterations'].append({'p_iter': p_iter, 'bic': bics[p_iter],
                                             'graph': nx.to_dict_of_lists(graph), 
                                             'time': times[p_iter]})
    incoming_edges = setting['in_edges']
    p = setting['p']
    print(f'{incoming_edges} {p} done')

with open('results/edge_deletion_pc1_maxcond5.json', 'w') as f:
    json.dump(results, f)

2 5 done


In [ ]:
with open('results/edge_deletion.json', 'r') as f:
    results = json.load(f)

for setting in results:
    for experiment in setting['experiments']:
        experiment['iterations'] = []
        ts = pd.read_csv('results/'+experiment['data'])

        _, graphs, times, bics, sepsets, cond_sizes = pc_incremental_pc1(
            partial_corr_test, ts, setting['alpha'], setting['p_max'], 
            verbose=True, patiency=setting['patience'])
        experiment['sepsets'] = [{'from': k[0], 'to': k[1], 'sepset': v} for k,v in sepsets.items()]
        experiment['cond_sizes'] = cond_sizes
        for p_iter, graph in graphs.items():
            experiment['iterations'].append({'p_iter': p_iter, 'bic': bics[p_iter],
                                             'graph': nx.to_dict_of_lists(graph), 
                                             'time': times[p_iter]})
    incoming_edges = setting['in_edges']
    p = setting['p']
    print(f'{incoming_edges} {p} done')

with open('results/edge_deletion_pc1.json', 'w') as f:
    json.dump(results, f)

In [ ]:
with open('results/edge_deletion.json', 'r') as f:
    results = json.load(f)

for setting in results:
    for experiment in setting['experiments']:
        experiment['iterations'] = []
        ts = pd.read_csv('results/'+experiment['data'])

        _, graphs, times, bics, sepsets, cond_sizes = pc_incremental_pc1(partial_corr_test, ts, setting['alpha'], setting['p_max'], 
                                                verbose=True, patiency=setting['patience'], steps=setting['p'])
        experiment['sepsets'] = [{'from': k[0], 'to': k[1], 'sepset': v} for k,v in sepsets.items()]
        experiment['cond_sizes'] = cond_sizes
        for p_iter, graph in graphs.items():
            experiment['iterations'].append({'p_iter': p_iter, 'bic': bics[p_iter],
                                             'graph': nx.to_dict_of_lists(graph),
                                             'time': times[p_iter]})
    incoming_edges = setting['in_edges']
    p = setting['p']
    print(f'{incoming_edges} {p} done')

with open('results/edge_deletion_pc1_noniterative.json', 'w') as f:
    json.dump(results, f)